# 8. Automating Urban Data Visualisation

In this project, we are going to learn how to create an animated spatio-temporal urban data visualisation, in which the temporal dimension is broken down to weekdays and hours of the day.

In [1]:
import pandas as pd
import numpy as np
import time 

import folium
from folium.plugins import HeatMap
from IPython.display import IFrame

import os
import time
from selenium import webdriver
from PIL import Image 
import glob

# %install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

time: 0 ns (started: 2021-11-02 11:28:43 +04:00)


Let's load the [NYC taxi data set](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page) corresponding to yellow cab trip records from January 2015.

In [2]:
data_2015 = pd.read_csv("../data/yellow_tripdata_2015-01.csv")
data_2015.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


time: 1min 18s (started: 2021-11-02 11:28:43 +04:00)


In [3]:
print("Number of columns = "+str(len(data_2015.columns)))

Number of columns = 19
time: 0 ns (started: 2021-11-02 11:30:02 +04:00)


In [4]:
data_2015.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

time: 16 ms (started: 2021-11-02 11:30:02 +04:00)


Now, let's slice our dataframe, extracting only those columns we need. Let's also convert the pickup times to `datetime` objects and create two new columns where we will store their values as weekdays and hours of the day.

In [5]:
df = data_2015[['tpep_pickup_datetime', 'pickup_longitude', 'pickup_latitude']].copy()
df['day_of_week'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.dayofweek
df['hour_of_day'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.hour

time: 20.3 s (started: 2021-11-02 11:30:02 +04:00)


In [6]:
df.head()

,tpep_pickup_datetime,pickup_longitude,pickup_latitude,day_of_week,hour_of_day
0,2015-01-15 19:05:39,-73.993896,40.750111,3,19
1,2015-01-10 20:33:38,-74.001648,40.724243,5,20
2,2015-01-10 20:33:38,-73.963341,40.802788,5,20
3,2015-01-10 20:33:39,-74.009087,40.713818,5,20
4,2015-01-10 20:33:39,-73.971176,40.762428,5,20


time: 0 ns (started: 2021-11-02 11:30:22 +04:00)


Let's now define a function that saves and visualises a map on a web browser.

In [7]:
# Function to visualize map
def save_visualise_map(map, filename):
    map.save(filename)
    return IFrame(filename, width='100%', height='600px')

time: 78 ms (started: 2021-11-02 11:30:22 +04:00)


Then, let's create a dictionary containing the encoding for the days of the week.

In [8]:
weekday_dict = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

time: 47 ms (started: 2021-11-02 11:30:22 +04:00)


Now, let's create a directory called `html_maps_pickup` in which we will save the `.html` files containing the interactive spatial visualisations of our GPS data for each day of the week and for each hour of the day.

In [9]:
from tqdm import tqdm
for i in tqdm(range(df.day_of_week.min(), df.day_of_week.max()+1)):
    for j in range(df.hour_of_day.min(), df.hour_of_day.max()+1):    
        
        # Filter to include only data for each day of week and hour of day
        df_geo = df.loc[(df.day_of_week==i) & (df.hour_of_day==j)][['pickup_latitude', 'pickup_longitude']].copy()

        # Instantiate map object 
        map_ = folium.Map(location=[40.75, -73.96], tiles='openstreetmap', zoom_start=12)

        # Plot heatmap
        HeatMap(data=df_geo, radius=12).add_to(map_)

        # Get day of week string from weekday_dict
        d = weekday_dict.get(i)
        
        # Add title to heatmap
        title_html = f'''<h3 align="center" style="font-size:20px">
                        <b>NYC Cab Pickups at {j}:00 on {d}: {len(df_geo)} rides</b></h3>
                     '''
        map_.get_root().html.add_child(folium.Element(title_html))

        #Save map
        save_visualise_map(map_, f'./html_maps_pickup/{i}_{j}_heatmap.html')

100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [04:05<00:00, 35.11s/it]

time: 4min 6s (started: 2021-11-02 11:30:23 +04:00)


Finally, let's take advantage of the `selenium` API to automate the screenshot taking process:

In [10]:
from webdriver_manager.chrome import ChromeDriverManager

for i in range(0, 7):
    for j in range(0, 24):
        # Set file path
        tmpurl=f'C:/Users/gevor/Downloads/Urban Data Science Course/Notebooks/html_maps_pickup//{i}_{j}_heatmap.html'
        
        # Set browser to Chrome
        browser = webdriver.Chrome(ChromeDriverManager().install())
        
        # Open file in browser
        browser.get(tmpurl)
        
        # If hour is < 10, add 0 for sorting purposes and to keep chronological order
        if j < 10:
            browser.save_screenshot(f'./maps_png_pickup/{i}_0{j}_heatmap.png')
        else:
            browser.save_screenshot(f'./maps_png_pickup/{i}_{j}_heatmap.png')
        
        # Close browser
        browser.quit()
    print("Day {} screenshots completed".format(i+1))

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.54/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54]
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Curre

Day 1 screenshots completed


[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATE

Day 2 screenshots completed


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\c

...

Day 5 screenshots completed


[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATE

Day 6 screenshots completed


[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\gevor\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATE

Day 7 screenshots completed
time: 15min 41s (started: 2021-11-02 11:34:29 +04:00)


Now that we have all screenshots, let's define a function that creates a GIF from this collection of taken screenshots.

In [11]:
def png_to_gif(path_to_images, save_file_path, duration=400):
    frames = []
    
    # Retrieve image files
    images = glob.glob(f'{path_to_images}')
    
    # Loop through image files to open, resize them and append them to frames
    for i in sorted(images): 
        im = Image.open(i)
        im = im.resize((825,584),Image.ANTIALIAS)
        frames.append(im.copy())
        
    # Save frames/ stitched images as .gif
    frames[0].save(f'{save_file_path}', format='GIF', append_images=frames[1:], save_all=True,
                   duration=duration, loop=0)

time: 0 ns (started: 2021-11-02 11:50:11 +04:00)


In [12]:
png_to_gif(path_to_images='./maps_png_pickup/*.png', 
           save_file_path='./automated urban data viz/NYC_pickup_heatmap.gif',
           duration=400)

time: 21.2 s (started: 2021-11-02 11:50:11 +04:00)


In [13]:
from IPython.display import HTML
HTML('<img src="./automated urban data viz/NYC_pickup_heatmap.gif">')

time: 0 ns (started: 2021-11-02 11:50:33 +04:00)


## Homework Assignment

Since you already know how to extract origins and destinations from raw GPS trajectories, complete the above automated data visualisation exercise for London, Rome, or, if you want a harder task, for the Yerevan GG data. Make sure to pick a temporal resolution and heatmap radius for best visual results.

Complete the task in a clean Jupyter Notebook, name the notebook **< Name > < Surname > Class 8 Homework** and upload it to Moodle.